<a href="https://colab.research.google.com/github/m-fila/uczenie-maszynowe-2021-22/blob/main/10_Sieci_głębokie_regularyzacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sieci neuronowe 
Autor: Anna Dawi

Korekta: A. Kalinowskid

Dzisiaj pod lupę weźmiemy jeden z najpopularniejszych i wszechstronnych modeli uczenia maszynowego - sieci neuronowe. Skupimy się na dwóch zagadnieniach: jak znaleźć optymalną architekturę sieci oraz o arcyważnym problemie regularyzacji. Będziemy pracować na zbiorze MNIST, czyli zbiorze czarno-białych obrazków z ręcznie napisanymi cyframi.

## Przygotowanie środowiska programistycznego

By zapewnić powtarzalność wyników ustawiany ziarno generatora liczb losowych:
```
seed = 128
rng = np.random.RandomState(seed)
```

In [ ]:
import sys, os

from termcolor import colored
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

import keras
from keras.datasets import mnist
from keras.utils import np_utils

import tensorflow as tf

seed = 128
rng = np.random.RandomState(seed)

def testModelOnMyDigits(model):    
    #Code created by M.Fila
    if not os.path.isdir("colab_freehands"):
        !git clone https://github.com/m-fila/colab_freehands.git

    from colab_freehands.canvas import Canvas  
    canvas = Canvas(line_width=2)
    example = (
        canvas.to_array(size=(20, 20), margin=(4, 4), dtype=np.float32, weighted=True) / 255
    )
    example_flatten = example.reshape( (-1))
    predictions = model(np.expand_dims(example_flatten, (0, -1)))
    plt.imshow(example, cmap="gray")
    plt.show()
    print(
        "Predicted class: {} ({:.0f}%)".format(
            np.argmax(predictions), np.max(predictions) * 100
        )
    )

## Import danych MNIST

Proszę:

* wczytać dane korzysatając z funkcji ```keras.datasets.mnist.load_data()```
* wypisać na ekran kształ danych uczących i testowych. Ile jest przykładów uczących i testowych? Jaką rozdzielczośc mają analizowane rysunki?
* korzystając z funkcji ```matplotlib.pyplot.imshow()``` narysować przykład numer 0 z danych uczących i wypisać jego etykietę

In [ ]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

print(colored("Training data features:", "blue"), ...)
print(colored("Training data labels:", "blue"), ...)

print(colored("Test data features:", "blue"), ...)
print(colored("Test data labels:", "blue"), ...)

print(colored("Trainig data example number 0:", "blue"), "label:", ...)
print(...)

plt.imshow(..., cmap='gray');

## Wstępne przygotowanie danych (ang. preprocessing)

Dane które analizujemy mają postać dwuwymiartowych macierzy, ale sieci neuronowe które dziś trenujemy przyjmują na wejściu jednowymiarowe wektory.
Proszę:

* zmienić kształ danych wejścionych na jednowymiarowe macierze - operacja "spłaszczenia"
* wypisać kształ macierzy po spłaszeniu. Czy wymiar jest zgodny z oczekiwaniem?
* znormalizować wartości danych do zakresu **[0,1]** korzysatając z funkcji ```numpy.amax(...)```

In [ ]:
print("Training data shape before flattening:", ...)

X_train = ...
X_test = ...

print("Training data shape after flattening:",...)

maxValue = ...
print("Maximum value in training data:",maxValue)

X_train = ...
X_test = ...

## Zmiana reprezentacji etykiet

Etykiety zawierają numer klasy - cyfry. Łatwiejszą do analizy postacią jest reprezentacja za pomocą słowa bitowego o długości równej licznie klas.
W takim słowie wszystkie bity, oprócz jednego - wsazującego na daną klasę mają wartość **0**:

```
Original label encoding: 5 shape: (60000,)
One hot label encoding: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] shape: (60000, 10)

```

Takie kodowanie nazywa się "one hot encoding".

Proszę:

* korzystając z funkcji ```tensorflow.one_hot(...)``` zamienić etykiety na reprezentację "one hot encoding"
* wypisać na ekran orginalne i nowe kodowanie etykiety dla przykładu 0 ze zbioru uczącego

In [ ]:
print("Original label encoding:",Y_train[0], "shape:", Y_train.shape)
depth = 10

Y_train = tf.one_hot(Y_train, depth)
Y_test = ...

print("One hot label encoding for training data:",..., "shape:", ...)
print("One hot label encoding for test data:",..., "shape:", ...)

## Definicja architektury sieci neuronowej

Proszę zdefiniować sieć neuronową o architekturze w pełni połączonej (ang. fully connected). Sieć powinna mieć:
* warstwę wejściową
* 4 warstwy ukryte o 500 neuronach każda z funkcją aktywacji ```relu``
* wartę wyjściową z funkcją aktywacji ```sofmax```  


Proszę:

* przeanalizować funkcję ```getModel(nHidden, nNeurons, inputWidth, outputWidth)``` która tworzy model o zadanej liczbe wartw ukrytych ```nHidden``` z zadaną liczbą neuronów w każdej warstwie ```nNeurons``` przyjmujący na wejściu wektor o długości ```inputWidth```
* obliczyć samodzielnie liczbę parametrów pierwszej warsty ukrytej i porównać ją z wynikiem działania funkcji ```model.summary(...)```
* obejrzeć rysunek przedstawiający architekturę modelu, uzysdkany przy pomocy funkcji ```tf.keras.utils.plot_model(...)```

In [ ]:
def getModel(nNeurons, nHiddenLayers, inputWidth, outputWidth):
      
    inputs = tf.keras.Input(shape=(inputWidth,))
    x = inputs
    for iHidden in range(nHiddenLayers):   
        x = tf.keras.layers.Dense(nNeurons, activation=tf.nn.relu)(x)
  
    outputs = tf.keras.layers.Dense(outputWidth, activation=tf.nn.softmax)(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

nNeurons = 500
nHiddenLayers = 5
inputWidth = 28*28
outputWidth = 10
model_basic = getModel(nNeurons, nHiddenLayers, inputWidth, outputWidth)

print(colored("Number of parameters for the first hidden layer:","blue"),inputWidth*nNeurons+nNeurons)

model_basic.summary()
tf.keras.utils.plot_model(model_basic, 'ML_model.png', show_shapes=True)

## Trenowanie modelu

Trenowanie modelu polega na znalezienu parametrów modelu dla których funkcja straty przyjmuje minimalną wartość. Problem ten w ogólności nie może być rozwiązany analiztycznie i minimalizacja jest przeprowadzana numerycznie. Standardowym wyborem algorytmu minimalizacji jest 
[adaptive momemtum estimation](https://arxiv.org/abs/1412.6980). W sytuaccji kiedy model ma za zadanie kategoryzację danych jako funkcję straty przyjmuje się zwykle entropię krzyżową (ang. crossentropy). W sytuacji kiedy klas jest więcej niż dwie trzeba użyć wariantu "categorical_crossentropy".


Trening jest prowadzony iteracyjnie. Każde przejście przez pełen zestaw danych jest **epoką**. Dane zwykle są podzielone na paczki **batch**.
Postęp treningu można monitorować używająć wybranej metryki na danych uczących i testowych. Zwykle używa się dokładności.

Proszę:

* przeprowadzić trening dla `10` epok z rozmiarem paczki wynoszącym `128`
* jako algorytmu minimalizacji proszę użyć `adam`
* jako funkcji straty proszę użyć `categorical_crossentropy`
* jako metryki proszę użyć `accuracy`

**Uwaga**: po pierwszym treningu proszę pełączyć śrdowisko wykonawcze by używało karty graficznej "GPU": z menu na górze:
```
Środowisko wykonawcze -> Zmień typ środowiska wykonawczego
```
Oczekiwany efekt:
```
Epoch 1/10
469/469 [==============================] - 2s 3ms/step - loss: 0.2286 - accuracy: 0.9302 - val_loss: 0.1268 - val_accuracy: 0.9619

...

Epoch 10/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0233 - accuracy: 0.9930 - val_loss: 0.0861 - val_accuracy: 0.9803
```

Proszę:
* porównać wartości metryki na danych uczących i testowych. Jaki wniosek wynika z tego porównania?

In [ ]:
%%time 

epochs = 15
batch_size = 128

model_basic_fit = model_basic.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test))

## Analiza historii treningu

Proszę uzupełnić funkcję ```plotTrainingHistory(model)``` tak by tworzyła wykresy:

* na jednym wykresie wartości metryki w funkcji numeru epoki obliczone dla danych uczących i treningowych
* na drugim wykresie funkcji straty  w zależności od numeru epoki obliczone dla danych uczących i treningowych

Wartości potrzebnych parametrów są dostępne w obiekcie ```Model.history```

In [ ]:
print(model_basic_fit.history.keys())
history = model_basic_fit.history
print(history['accuracy'])

def plotTrainingHistory(model):

    fig, axes= plt.subplots(1,2,figsize=(10,5))
    history = model.history
    axes[0].plot(...)
    axes[0].plot(...)
    axes[0].set_ylabel(...)
    axes[0].set_xlabel(...)
    axes[0].legend(['train', 'validation'], loc='upper left')

    axes[1]...
    ...
    ...
    
    
plotTrainingHistory(model_basic_fit)   

## Sprawdzenie modelu na własnych danych

Proszę uruchomić komórkę poniżej i sprawdzić jak model rozpoznaje własnoręcznie napisane cyfry

**Uwaga** ten fragment działa tylko w Google Colaboratory

In [ ]:
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB: 
    testModelOnMyDigits(model_basic)

## Regularyzacja procesu treningu

Analiza historii uczenia i wartości metryk na zbiorahc uczącym i treninsgowym wskazuje, że uzyskany model nie generalizuje się dobrze - jego odpowiedź na dane których nie "widział" w czasie treningu jest gorsza niż na dane uczące. Jednym ze sposobów redukcji tego efektu jest **regularyzacja** modelu poprzez nałożeniu 
różnych ograqniczeń na proces uczenia to mogą być:

* zmiana liczby epok i zatrzymanie uczenia w chwili gdy metryka na zbiorze testowym się nie zmienia, lub pogarsza - ang. **early stopping**
* ograniczenia na wartości wag uzyskane przez dodawanie członów do funkcji straty kierujących prosec uczenia to wybranego obszaru wag, np. małych wartości wag - **L1 or L2 penalty term**
* losowe wyłączanie neuronów - ang. **dropout**

### Early stopping

Proszę:

* stworzyć model ```model_early_stop``` o tej samej architekturze co model ```model_basic```
* przeprowadzić trening z opcją "early_stop": ```callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]``` Co oznacza parametr "cierpliwość"?
* narysować rysunki dla historii treningu

Czy moment przerwania treningu jest zgodny z oczekiwaniem?

In [ ]:
%%time

model_early_stop = ...

model_early_stop_fit = model_early_stop.fit(...)

plotTrainingHistory(...) 

### Regularyzacja L2 i L1

Regularyzacje L1 i L2 prowadzą do uzyskania moeli które mają małe wartości wag. Efekty ten uzyskuje się przez dodanie członu "kary" ang. "penalty term" do funkcji straty:
 
\begin{equation}
L_{1} = \lambda \sum_{i} w_{i},~~
L_{2} = \lambda \sum_{i} w_{i}^{2}
\end{equation}

gdzie $w_{i}$ to wagi modelu, a $\lambda$ to parametr skalujący wielkość członu kary.

Proszę:

* stworzyć model ```model_L2``` o tej samej architekturze co model ```model_basic``` z regularyzacją L2 dla każdej warstwy. Efekt ten możne uzyskać na conajmniej dwa sposoby:
    * napisać nową funkcję ```getModelWithL2(...)``` z opcją regularyzacji L2 dla każdej warstwy: ```kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)```
    * dodać regularyzację dla warstw istniejącego modelu:
    
      ```
      layers = model_basic.layers
      ...
      layer.kernel_regularizer =  tf.keras.regularizers.l2(l2_lambda)
      ```
      W tym wypadku należy skompilowac model powtórnie:
      ```
      model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      ```
* przeprowadzić trening z parametrami jak poprzednio
* narysować rysunki dla historii treningu

In [ ]:
l2_lambda = 0.001

model_L2 = ...

...
 
model_L2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_L2_fit = model_L2.fit(...)
plotTrainingHistory(...)   

### Dropout

Opuszczanie neuronów (ang. dropout) polega na dezaktywacji losowych neutronów w czasie treningu. Dezaktywacja neuronów jesty uzyskiwane przez zerowanie wagi danwego neuronu względem następnej wartwy. Efekt ten uzyskuje się przez wstawienie dedykowanerj warstwy - "tf.keras.Dropout" po warstwie w której mają być dezaktywowane neurony.

Proszę:

* napisać funkcję ```getModelWithDropout(nNeurons, nHiddenLayers, inputWidth, outputWidth, dropout_rate)``` która tworzy model z warstawmi "Dropout" po każdej warsdtwie ukrytej
* stworzyć model ```model_droput``` o tej samej architekturze co model ```model_basic``` z warstwami "Dropout" po każdej warstwie ukrytej.
* wypisać na ekran architekturę modelu, używając funkcji ```model.summary()``` i sprawdzić czy struktura modelu jest zgodna z oczekiwaniem
* przeprowadzić trening z parametrami jak poprzednio
* narysować rysunki dla historii treningu

In [ ]:
def getModelWithDropout(nNeurons, nHiddenLayers, inputWidth, outputWidth, dropout_rate):
      
    ...
    ...
    return model

dropout_rate = 0.25
model_dropout = ...
model_dropout.summary()

model_dropout_fit = model_dropout.fit(...)
plotTrainingHistory(...) 

### Połączenie metod L2 i dropout

Proszę:

* napisać funkcję ```getModelWithDropoutWithL2(nNeurons, nHiddenLayers, inputWidth, outputWidth, dropout_rate, l2_lambda)``` która tworzy model z warstawmi "Dropout" po każdej warstwie ukrytej, oraz regularyzacją typu L2 o tej samej architekturze co model ```model_basic```
* stworzyć model ```model_droput_L2```
* wypisać na ekran architekturę modelu, używając funkcji ```model.summary()``` i sprawdzić czy struktura modelu jest zgodna z oczekiwaniem
* przeprowadzić trening z parametrami jak poprzednio
* narysować rysunki dla historii treningu. 
* **proszę przeanalizować i skomentować wykresy**. Czy są one zgodne wykresami dla innych modeli?

In [ ]:
def getModelWithDropoutWithL2(nNeurons, nHiddenLayers, inputWidth, outputWidth, dropout_rate, l2_lambda):
      
    ...
    return model

...
...
...

### Walidacja modelu

Który z wariantów jest optymalny? Czy mamy wystarczające dane by odpowiedzieć na to pytanie? 

Proszę:

* korzystająć z funkcji ```sklearn.model_selection.train_test_split(...)``` podzielić zbiór uczący na nowy uczący i walidacyjny w stosunku 7:3
* korzystając ze zbiorów uczącego i walidacyjnego wybrać optymalny model
* przeprowadzić trening modeli: `basic`, `early_stop`, `L2` i `L2_dropout` sprawdzając wydajność modelu na zbiorze walidacyjnym. W czasie treningu można zmiejszyć ilośc wypisywanych danych używając parametru ```verbose=2```
* wybrać model o najlepszej dokładności
* sprawdzić dokładność wybranego modelu na zbiorze testowym

In [ ]:
from sklearn.model_selection import ...

#Reload the data
...

#Check the shapes
...

#Flatten the data
...

#Normalise the data
...

#Recode the labels
...

#Traing the models
...

print(colored("Best model:","blue"))
...

print(colored("Best model metric on test data:","blue"))
...

## Zadanie domowe

### Warstawa spłaszczająca

Pierwsza operacja wstępnej analizy danych polegała na spłaszczeniu wielowymiarowje struktury do jednowymiarowego wektora. To jest częsta operacja i zdefiniowano dla niej odpowiednią warstwę (ang. layer).

Proszę:

* korzystająć z warstwy spłasznapisać funkcję ```getModelFinal(...)``` która tworzy wszystkim elementami (regularyzacja L2, dropout) który na wejściu przyjmuje oryginalne rysunki

In [ ]:
def getModelFinal(nNeurons, nHiddenLayers, inputWidth, outputWidth, dropout_rate, l2_lambda):
      
    ...
    return model

dropout_rate = 0.25
model_final = ...


#Reload the data
...

#Normalise the data
...

#Recode the labels
...


model_final = model_final.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), verbose=2)
plotTrainingHistory(model_final) 